https://pypi.org/project/requests-oauth2client/ -- Check for oaut2client documentation


In [55]:
import credentials as c
import json
import math
import pandas as pd

from requests_oauth2client import *

def get_json_response(api, endpoint):
    # Sends a GET request to the API and returns the parsed JSON response
    response = api.get(endpoint)
    response_data = response.content.decode('utf-8')
    return json.loads(response_data)

def calculate_num_pages(count, per_page):
    # Calculates the number of pages based on the total count and per page limit
    return math.ceil(count / per_page)

# OAuth2 authentication setup
token_endpoint = c.token_endpoint
client_id = c.client_id
client_secret = c.client_secret
api_url = c.api_url
scope = c.scope

oauth2client = OAuth2Client(
    token_endpoint=token_endpoint,
    auth=(client_id, client_secret),
)

auth = OAuth2ClientCredentialsAuth(
    oauth2client, scope=scope, resource=api_url
)

api = ApiClient(
    api_url, auth=auth
)

# Get total client count and per page limit
clients_response = get_json_response(api, "/clients")
clients_data = clients_response['data']
total_count = clients_data['info']['count']
per_page = clients_data['info']['per_page']

# Calculate number of pages
num_pages = calculate_num_pages(total_count, per_page)

# List to store all the clients records
clients = []

# Iterate over each page
for page in range(1, num_pages + 1):
    clients_response = get_json_response(api, f"/clients?page={page}")
    page_data = clients_response['data']

    if page_data is not None:
        for client in page_data['clients']:
            group_name = client['group']['name'] if client['group'] is not None else None

            client_info = [
                client['number'],
                client['email'],
                client['name'],
                client['surname'],
                client['phone'],
                client['company'],
                client['nip'],
                group_name,
                client['additional']['1'],
                client['additional']['2'],
                client['additional']['3']
            ]
            clients.append(client_info)
# typo#
attachmenst = []
rewards = []
operations = []

# Iterate over each client
for client in clients:
    # Get attachments count
    attachments_response = get_json_response(api, f"/clients/{client[0]}/attachments")
    attachments_data = attachments_response['data']
    attachments_count = attachments_data['info']['count']
    # typo #
    attachmenst.append([client[0], attachments_count])

    # Get reward
    rewards_response = get_json_response(api, f"/clients/{client[0]}/rewards/orders")
    rewards_data = rewards_response['data']
    rewards_count = rewards_data['info']['count']
    rewards_per_page = rewards_data['info']['per_page']
    rewards_num_pages = calculate_num_pages(rewards_count, rewards_per_page)

    for page in range(1, rewards_num_pages + 1):
        rewards_response = get_json_response(api, f"/clients/{client[0]}/rewards/orders")
        rewards.extend([
            [
                reward_order['client']['number'],
                reward_order['reward']['name'],
                reward_order['quantity'],
                reward_order['points'],
                reward_order['status'],
                reward_order['date_add']
            ]
            for reward_order in rewards_response['data']['rewards_orders']
        ])

    # Get operations
    operations_response = get_json_response(api, f"/clients/{client[0]}/operations")
    operations_data = operations_response['data']
    operations_count = operations_data['info']['count']
    operations_per_page = operations_data['info']['per_page']
    operations_num_pages = calculate_num_pages(operations_count, operations_per_page)

    for page in range(1, operations_num_pages + 1):
        operations_response = get_json_response(api, f"/clients/{client[0]}/operations?page={page}")
        operations.extend([
                [
                    operation['client']['number'],
                    operation['type'],
                    operation['date_add'],
                    operation['tags'],
                    operation['description']
                ]
                for operation in operations_response['data']['operations']
        ])

In [61]:
import csv

# Save clients list to a CSV file with UTF-8 encoding
with open('clients.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(clients)

# Save attachments list to a CSV file with UTF-8 encoding
with open('attachments.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(attachments)

# Save rewards list to a CSV file with UTF-8 encoding
with open('rewards.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(rewards)

# Save operations list to a CSV file with UTF-8 encoding
with open('operations.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(operations)


In [3]:
import csv

# Load clients from CSV file with UTF-8 encoding
with open('clients.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    clients = list(reader)

# Load attachments from CSV file with UTF-8 encoding
with open('attachments.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    attachments = list(reader)

# Load rewards from CSV file with UTF-8 encoding
with open('rewards.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    rewards = list(reader)

# Load operations from CSV file with UTF-8 encoding
with open('operations.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    operations = list(reader)


Creating excel files

In [4]:
import pandas as pd


clients_columns = ['client_id', 'client_email', 'client_name', 'client_surname', 'client_phone', 'company', 'client_nip', 'account_name', 'salesrep', 'products', 'delivery']
clients_df = pd.DataFrame(clients)
clients_df.columns = clients_columns
clients_df['client_id'] = 'BB_' + clients_df['client_id'] 
clients_df['salesrep'] = clients_df['salesrep'].str.split(' - ').str[0]
clients_df.to_excel('output/sellout_clients.xlsx', index=False)

In [5]:
attachments_columns = ['client_id', 'has_attachment']
attachments_df = pd.DataFrame(attachments)
attachments_df.columns = attachments_columns
attachments_df['client_id'] = 'BB_' + attachments_df['client_id']
attachments_df['has_attachment'] = attachments_df['has_attachment'].apply(lambda x: 1 if int(x) > 0 else 0)
attachments_df.to_excel('output/sellout_attachments.xlsx', index=False)

In [6]:
rewards_columns = ['client_id', 'reward_name', 'quantity', 'points', 'status', 'date_add']
rewards_df = pd.DataFrame(rewards)
rewards_df.columns = rewards_columns
rewards_df['client_id'] = 'BB_' + rewards_df['client_id']
rewards_df.to_excel('output/rewards.xlsx', index=False)

In [8]:
operations_columns = ['client_id', 'type', 'date_add', 'tag', 'description']
operations_df = pd.DataFrame(operations)
operations_df.columns = operations_columns
operations_df['client_id'] = 'BB_' + operations_df['client_id']
#operations_df.to_excel('output/operations.xlsx', index=False)

# Drop rows where tag is empty
operations_df = operations_df[operations_df['tag'] != '']

In [ ]:
new_rows = []
for _, row in operations_df.iterrows():
    client_id = row['client_id']
    operation = row['type']
    date = row['date_add']
    description = row['description']

    products = description.split(',')
    for product in products:
        parts = product.split(' - ') # by last 
        if len(parts) == 2:
            product_info = parts[0].strip().split('x') # by first x
            if len(product_info) == 2:
                quantity = product_info[0].strip()
                product_name = product_info[1].strip()
                product_id = parts[1].strip()

                new_row = [client_id, operation, date, product_id, product_name, quantity]
                new_rows.append(new_row)

products_df = pd.DataFrame(new_rows, columns=['client_id', 'operation', 'date', 'product_id', 'product_name', 'quantity'])

products_df.to_excel('output/products.xlsx', index=False)

In [12]:
line = "5xAlugastrin 3 Forte tabletki  - 9095729,7xManusan 500 ml - 8608311,9xUrgo Popękana skóra dłonie i stopy (Filmogel) - 9088905,1xUrgo Stop obgryzaniu paznokci (Filmogel) - 7087337,1xUrgo Zniszczone paznokcie przez grzybicę i urazy (Filmogel) - 9102487,7xUrgo Oparzenia i rany powierzchniowe - 9105556,2xURGO Resistant taśma 1 m x 6 cm - 9104187,1xURGO Resistant opatrunki 20 szt. - 9096186,1xURGO Szybkie gojenie ran - 9070760,3xURGO Urgosterile 5x9 - 9103497,1xURGO URGOSTERILE 20X9  - 9094769,1xEau Thermale Jedwabisty balsam do ciała  - 7062109,2xBariederm Cica-Daily serum odbudowujące  - 7083274,2xBaiederm Cica-Daily żel krem odbudowujący  - 7083273,1xBariederm Balsam do skóry popękanej - 7066893,4xBariederm Balsam do spierzchniętych ust - 7066892,2xBariederm Dermatologiczny CICA-olejek  - 7080597,1xBariederm CICA - spray osuszajaco-regenerujący  - 7080985,3xBariederm CICA - Krem regenerująco-ochronny 100ml - 7066880,5xBariederm CICA - Krem regenerująco-ochronny 40ml - 7066879,3xBariederm regenerująco-ochronny krem do rąk - 7066881,1xBÉBE 1st MLECZKO NAWILŻAJĄCE DLA DZIECI 500ML  - 7085385,1xBÉBE 1st WODA OCZYSZCZAJĄCA DLA DZIECI 500ML  - 7085376,1xCreme Lavante Krem do mycia 500ml - 7039759,1xDS Żel myjący - 7050633,1xDS HAIR Płyn - 7039762,1xDezodorant roll-on - 7055277,1xBariederm regenerująco-ochronny krem do rąk - 7066881,2xEau Thermale Aktywnie nawiżający krem do rąk - 7069052,5xEau Thermale nawilżajaco-odżywcza pomadka do ust z WTU  - 7081626,2xHyseac Pasta SOS - 7059229,1xHyseac serum redukujące niedoskonałości  - 7081614,1xHyseac  Żel oczyszczający 150ml - 7039663,3xEau Thermale Aktywnie nawilżająca maseczka na noc  - 7071965,3xOlejek pod prysznic - 7069435,3xRoseliane krem do cery naczynkowej - 7041406,2xWoda termalna Uriage 50 ml - 7044045,4xWoda termalna Uriage 150 ml - 7039674,3xWoda termalna Uriage 300 ml - 7046743,1xXemose CERAT Intensywny balsam do skóry bardzo suchej - 7061554,2xXemose kojący krem pod oczy  - 7082192,4xXemose Krem do twarzy - 7051273,7xXemose Krem do skóry bardzo suchej - 7039717,4xXemose olejek do kąpieli - 7054470,1xXemose Kojący olejek w kremie 500 ml  - 7078698,1xXemose Nawilżająco-ochronna pomadka do ust  - 7075536,12xURGO Urgostrips - 9105259"
products = line.split(',')
for product in products:
    parts = product.split(' - ') # by last ' - '
    if len(parts)

In [13]:
products

['5xAlugastrin 3 Forte tabletki  - 9095729',
 '7xManusan 500 ml - 8608311',
 '9xUrgo Popękana skóra dłonie i stopy (Filmogel) - 9088905',
 '1xUrgo Stop obgryzaniu paznokci (Filmogel) - 7087337',
 '1xUrgo Zniszczone paznokcie przez grzybicę i urazy (Filmogel) - 9102487',
 '7xUrgo Oparzenia i rany powierzchniowe - 9105556',
 '2xURGO Resistant taśma 1 m x 6 cm - 9104187',
 '1xURGO Resistant opatrunki 20 szt. - 9096186',
 '1xURGO Szybkie gojenie ran - 9070760',
 '3xURGO Urgosterile 5x9 - 9103497',
 '1xURGO URGOSTERILE 20X9  - 9094769',
 '1xEau Thermale Jedwabisty balsam do ciała  - 7062109',
 '2xBariederm Cica-Daily serum odbudowujące  - 7083274',
 '2xBaiederm Cica-Daily żel krem odbudowujący  - 7083273',
 '1xBariederm Balsam do skóry popękanej - 7066893',
 '4xBariederm Balsam do spierzchniętych ust - 7066892',
 '2xBariederm Dermatologiczny CICA-olejek  - 7080597',
 '1xBariederm CICA - spray osuszajaco-regenerujący  - 7080985',
 '3xBariederm CICA - Krem regenerująco-ochronny 100ml - 706688

In [9]:
operations_df

,client_id,type,date_add,tag,description
4,BB_10116,add,2023-01-20 13:57:28,"Alugastrin 3 Forte tabletki - 9095729,Manusan...","5xAlugastrin 3 Forte tabletki - 9095729,7xMan..."
8,BB_10116,add,2022-07-21 14:59:52,"ImmunoStim (blister),Urgo Urgostrips,Manusan 5...","14xImmunoStim (blister),14xUrgo Urgostrips,8xM..."
9,BB_10116,add,2022-07-21 14:28:58,Magnez Organiczny + B6,47xMagnez Organiczny + B6
14,BB_10151,add,2023-04-21 11:35:46,"Alugastrin 3 Forte tab. - 9095729,Bariederm Ba...","25xAlugastrin 3 Forte tab. - 9095729,2xBariede..."
22,BB_10151,add,2023-04-16 12:34:36,"Alugastrin 3 Forte tab. - 9095729,Alugastrin 3...","7xAlugastrin 3 Forte tab. - 9095729,5xAlugastr..."
...,...,...,...,...,...
10525,BB_389805,add,2023-04-20 15:40:04,Laboratoria PolfaŁodź B-complex x 50 tabl. - ...,38xLaboratoria PolfaŁodź B-complex x 50 tabl. ...
10526,BB_389805,add,2023-04-20 14:48:07,Alugastrin 3 Forte tab. - 9095729,15xAlugastrin 3 Forte tab. - 9095729
10529,BB_244541,add,2023-04-20 14:36:48,"URGO Sensitive Stretch - 9101457,URGO Coricide...","3xURGO Sensitive Stretch - 9101457,2xURGO Cori..."
10531,BB_344568,add,2023-04-21 07:34:26,"URGO Popękana skóra dłonie i stopy - 9088905,...",6xURGO Popękana skóra dłonie i stopy - 908890...
